In [1]:
from dotenv import load_dotenv, find_dotenv
from pymongo import MongoClient
import os
import pprint

1. Cargamos las variables que contienen la informacion del Logeo

In [2]:
load_dotenv(find_dotenv())
PUERTO = os.environ.get('MONGO_PORT')

2. Creamos la conexion a MongoDB y listamos las BD

In [3]:
client = MongoClient(f"mongodb://localhost:{PUERTO}")
print(client.list_database_names())

['admin', 'config', 'local', 'miDB']


3. Trabajar con una BD especifica

In [4]:
miDB = client.get_database('miDB')
#print(list(miDB.list_collections()))
#print(miDB.list_collection_names())

4. INSERT 

Ejemplo de INSERT de un documento dentro de una collection

4.1 Insert One

In [26]:
def insert_test_doc():
    collection = miDB.alumnos
    test_doc = {
        "name":"Nicolas",
        "type":"Test"
    }
    insert_id = collection.insert_one(test_doc).inserted_id
    print(insert_id)

insert_test_doc()

65dc983183484190e2657086


4.2 Insert Many

In [35]:
def create_document():
    collection = miDB.alumnos
    nombre = ['Nicolas', 'Martin', 'Alejandro', 'Patricio']
    edad = [36,35,35,36]
    apellido = ['Leali','Caronte','Francolino','Perez']

    array = []
    for nombre, edad, apellido in zip(nombre, edad, apellido):
        doc = {'nombre':nombre, 'apellido':apellido, 'edad':edad}
        array.append(doc)
    
    collection.insert_many(array)
    
create_document()

5. SELECT

5.1 .find()

Busca todos los registros

In [8]:
printer = pprint.PrettyPrinter()

def find_all_people():
    collection = miDB.alumnos
    people = collection.find()
    
    for person in people:
        printer.pprint(person)
    
find_all_people()

{'_id': ObjectId('65da4b127579225f818ad35a'), 'edad': 36, 'name': 'nicolas'}
{'_id': ObjectId('65dc982b83484190e2657085'), 'name': 'Nicolas', 'type': 'Test'}
{'_id': ObjectId('65dc983183484190e2657086'), 'name': 'Nicolas', 'type': 'Test'}
{'_id': ObjectId('65dc9ec083484190e2657087'),
 'apellido': 'Leali',
 'edad': 36,
 'nombre': 'Nicolas'}
{'_id': ObjectId('65dc9ec083484190e2657088'),
 'apellido': 'Caronte',
 'edad': 35,
 'nombre': 'Martin'}
{'_id': ObjectId('65dc9ec083484190e2657089'),
 'apellido': 'Francolino',
 'edad': 35,
 'nombre': 'Alejandro'}
{'_id': ObjectId('65dc9ec083484190e265708a'),
 'apellido': 'Perez',
 'edad': 36,
 'nombre': 'Patricio'}


5.2 fin_one()

Lo usamos para encontrar un documento que cumple con el criterio de busqueda. Solo trae el primero.

In [14]:
printer = pprint.PrettyPrinter()
def find_nicolas():
    collection = miDB.alumnos
    nicolas = collection.find_one({'nombre':'Nicolas', 'edad':36})
    printer.pprint(nicolas)
    
find_nicolas()

{'_id': ObjectId('65dc9ec083484190e2657087'),
 'apellido': 'Leali',
 'edad': 36,
 'nombre': 'Nicolas'}


5.3 Count

In [18]:
def count_numero_of_people():
    collection = miDB.alumnos
    total = collection.count_documents(filter={})
    print(total)
    
count_numero_of_people()

7


5.4 Select by ID

In [28]:
printer = pprint.PrettyPrinter()

def get_person_by_id(id_person):
    collection = miDB.alumnos
    from bson.objectid import ObjectId

    _id = ObjectId(id_person)

    person = collection.find_one({'_id':_id})
    printer.pprint(person)
    
get_person_by_id('65dc9ec083484190e2657088')  

{'_id': ObjectId('65dc9ec083484190e2657088'),
 'apellido': 'Caronte',
 'edad': 35,
 'nombre': 'Martin'}


In [48]:
printer = pprint.PrettyPrinter()
def get_people_by_age(min_age, max_age):
    collection = miDB.alumnos
    query = {
            "$and":
            [
                {"edad":{"$gte":min_age}},
                {"edad":{"$lte":max_age}}
            ]
        }

    people = collection.find(query).sort("edad")
    for person in people:
        printer.pprint(person)

get_people_by_age(34,36)

{'_id': ObjectId('65dc9ec083484190e2657088'),
 'apellido': 'Caronte',
 'edad': 35,
 'nombre': 'Martin'}
{'_id': ObjectId('65dc9ec083484190e2657089'),
 'apellido': 'Francolino',
 'edad': 35,
 'nombre': 'Alejandro'}
{'_id': ObjectId('65da4b127579225f818ad35a'), 'edad': 36, 'name': 'nicolas'}
{'_id': ObjectId('65dc9ec083484190e2657087'),
 'apellido': 'Leali',
 'edad': 36,
 'nombre': 'Nicolas'}
{'_id': ObjectId('65dc9ec083484190e265708a'),
 'apellido': 'Perez',
 'edad': 36,
 'nombre': 'Patricio'}


5.5 Mostrar solo algunas columnas.

In [51]:
printer = pprint.PrettyPrinter()

def show_only_columns():
    columnas = {'_id':0, 'nombre':1, 'edad': 1}
    collection = miDB.alumnos

    personas = collection.find({}, columnas) 

    for persona in personas:
        printer.pprint(persona)

show_only_columns()

{'edad': 36}
{}
{}
{'edad': 36, 'nombre': 'Nicolas'}
{'edad': 35, 'nombre': 'Martin'}
{'edad': 35, 'nombre': 'Alejandro'}
{'edad': 36, 'nombre': 'Patricio'}


6. Update

6.1 Update

Agregamos un nuevo campo, incrementamos la edad y cambiamos el nombre de una columna

In [56]:
printer = pprint.PrettyPrinter()

def update_persona_by_id(id_per):
    from bson.objectid import ObjectId

    collection = miDB.alumnos

    sql = {
        "$set":{"new_field":True},
        "$inc":{"edad":10},
        "$rename":{"apellido":"last_name"}
    }

    _id = ObjectId(id_per)

    collection.update_one({"_id":_id}, sql)

update_persona_by_id('65dc9ec083484190e2657089')

6.2 Update  - delete one field

Usamos __$unset__ para eliminar un campo de un documento

In [62]:
printer = pprint.PrettyPrinter()

def update_persona_by_id(id_per):
    from bson.objectid import ObjectId

    _id = ObjectId(id_per)

    sql = {
        "$unset":{'new_field':''}
    }

    collection = miDB.alumnos

    collection.update_one({'_id':_id}, sql)

update_persona_by_id('65dc9ec083484190e2657089')

6.3 Update - Replace

Usamos replace cuando queremos conservar el _id pero cambiar todos los datos del documento

In [63]:
printer = pprint.PrettyPrinter()
def update_replace_persona(id_per):
    from bson.objectid import ObjectId

    _id = ObjectId(id_per)

    doc = {
        'Nombre':'Alejandro',
        'Apellido':'Francolino y Aguero',
        'Edad':37
    }
    
    collection = miDB.alumnos

    collection.replace_one({'_id':_id}, doc)

update_replace_persona('65dc9ec083484190e2657089') 

7. DELETE

In [64]:
printer = pprint.PrettyPrinter()

def delete_by_id(per_id):
    from bson.objectid import ObjectId
    
    _id = ObjectId(per_id)
    
    collection = miDB.alumnos
    
    collection.delete_one({'_id':_id})
    
delete_by_id('65dc9ec083484190e2657089')

8. RELACIONES en MongoDB

8.1 Embedings

In [71]:
mi_dict = {
                "street": "calle 01",
                "numero": 4567,
                "city": "CABA",
                "zip": 5643
            }

def add_address_embed(id_pers, address):
    from bson.objectid import ObjectId

    _id = ObjectId(id_pers)

    collection = miDB.alumnos

    SQL = {
            "$addToSet": {'addresses':address}
            }

    collection.update_one({"_id":_id}, SQL)

add_address_embed('65dc9ec083484190e265708a', mi_dict)

8.2 Foreign Keys

In [72]:
address = {
                "street": "calle 01",
                "numero": 4567,
                "city": "CABA",
                "zip": 5643
            }

def add_address_relation(per_id, address):
    from bson.objectid import ObjectId

    _id = ObjectId(per_id)

    address = address.copy()
    address["owner_id"] = _id

    address_collection = miDB.address
    address_collection.insert_one(address)

add_address_relation('65dc9ec083484190e2657087', address)